In [24]:
import numpy as np
import matplotlib.pyplot as plt

class NQueensProblem:
    def __init__(self, init_pop_size=100, no_of_queens=8, num_epochs=200, mutation_rate=0.1, crossover_rate=0.9):
        self.init_pop_size = init_pop_size
        self.no_of_queens = no_of_queens
        self.population = self.generate_population()
        self.num_epochs = num_epochs
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.best_metric = (self.no_of_queens * (self.no_of_queens - 1)) // 2

    def generate_individual(self):
        """Generates a valid individual with one queen per row."""
        individual = np.zeros((self.no_of_queens, self.no_of_queens), dtype=int)
        for i in range(self.no_of_queens):
            col = np.random.randint(0, self.no_of_queens)
            individual[i, col] = 1
        return individual


    def generate_population(self):
        """Generates the initial population."""
        return [self.generate_individual() for _ in range(self.init_pop_size)]

    def fitness_function(self, individual):
        """Calculates fitness score based on non-conflicting queen pairs."""
        queen_positions = [(i, row.tolist().index(1)) for i, row in enumerate(individual)]
        non_conflicting_pairs = self.calc_total_non_conflicting_pairs(queen_positions)
        return non_conflicting_pairs / self.best_metric

    def calc_total_non_conflicting_pairs(self, positions):
        """Calculates total non-conflicting pairs of queens."""
        total_pairs = len(positions) * (len(positions) - 1) // 2
        conflicts = sum(
            1 for i, (r1, c1) in enumerate(positions)
            for r2, c2 in positions[i + 1:]
            if r1 == r2 or c1 == c2 or abs(r1 - r2) == abs(c1 - c2)
        )
        return total_pairs - conflicts

    def mutate_children(self, individual):
        """Mutates an individual by randomly moving a queen in a row."""
        for row in range(self.no_of_queens):
            if np.random.uniform(0, 1) < self.mutation_rate:
                new_col = np.random.randint(0, self.no_of_queens)
                individual[row] = np.zeros(self.no_of_queens, dtype=int)
                individual[row][new_col] = 1
        return individual

        ''' if np.random.uniform(0, 1) < self.mutation_rate:
            row = np.random.randint(0, self.no_of_queens)
            new_col = np.random.randint(0, self.no_of_queens)
            individual[row] = np.zeros(self.no_of_queens, dtype=int)
            individual[row][new_col] = 1
        return individual
        '''

    def crossover(self, p1, p2):
        """Performs crossover to generate two offspring."""
        crossover_point = np.random.randint(1, self.no_of_queens)
        child1 = np.vstack((p1[:crossover_point], p2[crossover_point:]))
        child2 = np.vstack((p2[:crossover_point], p1[crossover_point:]))
        return child1, child2





    def run_mu_plus_lambda(self):
        print("Running N-Queens Problem with Mu + Lambda Selection: \n")

        #fitness_array = []
        fitness_progress = []


        for epoch in range(self.num_epochs):
            fitness_scores = [self.fitness_function(ind) for ind in self.population]
            best_fitness = max(fitness_scores)
            fitness_progress.append(best_fitness)

            print(f"Epoch {epoch + 1}: Best Fitness = {best_fitness:.4f}")

            # Sort population by fitness
            sorted_indices = sorted(range(len(fitness_scores)), key=lambda x: fitness_scores[x], reverse=True)
            best, second_best = sorted_indices[:2]

            # Generate offspring
            c1, c2 = self.crossover(self.population[best], self.population[second_best])
            mutated_c1 = self.mutate_children(c1)
            mutated_c2 = self.mutate_children(c2)

            # Append mutated and unmutated offspring to the population
            self.population.extend([mutated_c1, mutated_c2, c1, c2])

            # Limit the population size to μ (initial population size)
            combined_fitness_scores = [self.fitness_function(ind) for ind in self.population]
            sorted_indices = sorted(range(len(combined_fitness_scores)), key=lambda x: combined_fitness_scores[x], reverse=True)
            self.population = [self.population[i] for i in sorted_indices[:self.init_pop_size]]

            if best_fitness == 1.0:
                print("Solution found!")
                return self.population[best], fitness_progress

        print("No solution found. Best attempt:\n")
        print("Best individual: ", self.population[sorted_indices[0]])
        return self.population[sorted_indices[0]], fitness_progress

    def run_mu_lambda(self):
        """Runs the genetic algorithm with μ, λ selection."""
        print("Running N-Queens Genetic Algorithm with μ, λ selection ...")

        fitness_progress = []

        for epoch in range(self.num_epochs):
            # Calculate fitness for current population
            fitness_scores = [self.fitness_function(ind) for ind in self.population]
            best_fitness = max(fitness_scores)
            fitness_progress.append(best_fitness)

            print(f"Epoch {epoch + 1}: Best Fitness = {best_fitness:.4f}")

            # Select the two best individuals as parents
            sorted_indices = sorted(range(len(fitness_scores)), key=lambda x: fitness_scores[x], reverse=True)
            best, second_best = sorted_indices[:2]

            # Generate a new population of size init_pop_size (λ = 10 in this case)
            new_population = []
            while len(new_population) < self.init_pop_size:
                # Perform crossover
                c1, c2 = self.crossover(self.population[best], self.population[second_best])

                # Apply mutation
                mutated_c1 = self.mutate_children(c1)
                mutated_c2 = self.mutate_children(c2)

                # Add children to the new population
                new_population.append(mutated_c1)
                if len(new_population) < self.init_pop_size:  # Ensure we don't exceed the size
                    new_population.append(mutated_c2)

            # Replace the old population with the new population
            self.population = new_population

            # Early stopping if solution is found
            if best_fitness == 1.0:
                print("Solution found!")
                return self.population[sorted_indices[0]], fitness_progress

        print("No solution found. Best attempt:")
        print("Best individual: ", self.population[sorted_indices[0]])
        return self.population[sorted_indices[0]], fitness_progress



In [21]:
nqueens = NQueensProblem()
population, fitness_progress = nqueens.run_mu_lambda_changed()

Running N-Queens Genetic Algorithm with μ, λ selection ...
Epoch 1: Best Fitness = 0.9286
Epoch 2: Best Fitness = 0.9286
Epoch 3: Best Fitness = 0.9286
Epoch 4: Best Fitness = 0.9286
Epoch 5: Best Fitness = 0.9286
Epoch 6: Best Fitness = 0.9286
Epoch 7: Best Fitness = 0.9286
Epoch 8: Best Fitness = 0.9286
Epoch 9: Best Fitness = 0.9286
Epoch 10: Best Fitness = 0.9286
Epoch 11: Best Fitness = 0.9286
Epoch 12: Best Fitness = 0.9286
Epoch 13: Best Fitness = 0.9286
Epoch 14: Best Fitness = 0.9286
Epoch 15: Best Fitness = 0.9286
Epoch 16: Best Fitness = 0.9286
Epoch 17: Best Fitness = 0.9286
Epoch 18: Best Fitness = 0.9286
Epoch 19: Best Fitness = 0.9286
Epoch 20: Best Fitness = 0.9286
Epoch 21: Best Fitness = 0.9286
Epoch 22: Best Fitness = 0.9286
Epoch 23: Best Fitness = 0.9286
Epoch 24: Best Fitness = 0.9286
Epoch 25: Best Fitness = 0.9286
Epoch 26: Best Fitness = 0.9286
Epoch 27: Best Fitness = 0.9286
Epoch 28: Best Fitness = 0.9286
Epoch 29: Best Fitness = 0.9286
Epoch 30: Best Fitness

In [23]:
nqueens = NQueensProblem()
population, fitness_progress = nqueens.run_mu_lambda()

Running N-Queens Genetic Algorithm with μ, λ selection ...
Epoch 1: Best Fitness = 0.8571
Epoch 2: Best Fitness = 0.8571
Epoch 3: Best Fitness = 0.8571
Epoch 4: Best Fitness = 0.8571
Epoch 5: Best Fitness = 0.8571
Epoch 6: Best Fitness = 0.8571
Epoch 7: Best Fitness = 0.8571
Epoch 8: Best Fitness = 0.8571
Epoch 9: Best Fitness = 0.8571
Epoch 10: Best Fitness = 0.8571
Epoch 11: Best Fitness = 0.8571
Epoch 12: Best Fitness = 0.8571
Epoch 13: Best Fitness = 0.8571
Epoch 14: Best Fitness = 0.8571
Epoch 15: Best Fitness = 0.8571
Epoch 16: Best Fitness = 0.8571
Epoch 17: Best Fitness = 0.8571
Epoch 18: Best Fitness = 0.8571
Epoch 19: Best Fitness = 0.8571
Epoch 20: Best Fitness = 0.8571
Epoch 21: Best Fitness = 0.8571
Epoch 22: Best Fitness = 0.8571
Epoch 23: Best Fitness = 0.8571
Epoch 24: Best Fitness = 0.8571
Epoch 25: Best Fitness = 0.8571
Epoch 26: Best Fitness = 0.8571
Epoch 27: Best Fitness = 0.8571
Epoch 28: Best Fitness = 0.8571
Epoch 29: Best Fitness = 0.8571
Epoch 30: Best Fitness

In [22]:
nqueens = NQueensProblem()
population, fitness_progress = nqueens.run_mu_plus_lambda()


Running N-Queens Problem with Mu + Lambda Selection: 

Epoch 1: Best Fitness = 0.8571
Epoch 2: Best Fitness = 0.8571
Epoch 3: Best Fitness = 0.8571
Epoch 4: Best Fitness = 0.8571
Epoch 5: Best Fitness = 0.8571
Epoch 6: Best Fitness = 0.8929
Epoch 7: Best Fitness = 0.9286
Epoch 8: Best Fitness = 0.9286
Epoch 9: Best Fitness = 0.9286
Epoch 10: Best Fitness = 0.9286
Epoch 11: Best Fitness = 0.9286
Epoch 12: Best Fitness = 0.9286
Epoch 13: Best Fitness = 0.9286
Epoch 14: Best Fitness = 0.9286
Epoch 15: Best Fitness = 0.9286
Epoch 16: Best Fitness = 0.9286
Epoch 17: Best Fitness = 0.9643
Epoch 18: Best Fitness = 0.9643
Epoch 19: Best Fitness = 0.9643
Epoch 20: Best Fitness = 0.9643
Epoch 21: Best Fitness = 0.9643
Epoch 22: Best Fitness = 0.9643
Epoch 23: Best Fitness = 0.9643
Epoch 24: Best Fitness = 0.9643
Epoch 25: Best Fitness = 0.9643
Epoch 26: Best Fitness = 0.9643
Epoch 27: Best Fitness = 0.9643
Epoch 28: Best Fitness = 0.9643
Epoch 29: Best Fitness = 0.9643
Epoch 30: Best Fitness = 0

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9add33c0-a6d3-4fe2-9e64-cc02a9ff9347' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>